In [1]:
import json
import glob
import h_gen
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tq
import matplotlib.patches as patches
import numpy as np
from PIL import Image
import utils
import importlib
import meta
metadata = utils.load("./variables/metadata.p")
from keras.optimizers import *
from keras.models import Sequential
from keras.layers import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
import os
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
remote = callbacks.RemoteMonitor(root='http://localhost:9000')
from shutil import copyfile
import threading
import heatmaps_to_rect as htr
import pipeline_functions as pf
importlib.reload(htr)
importlib.reload(utils)
importlib.reload(meta)

Using Theano backend.
DEBUG: nvcc STDOUT nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
mod.cu
   Creating library C:/Users/yolo/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64/tmplfin_379/m91973e5c136ea49268a916ff971b7377.lib and object C:/Users/yolo/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64/tmplfin_379/m91973e5c136ea49268a916ff971b7377.exp

Using gpu device 0: GeForce GTX 970 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)
C:\Program Files\Anaconda3\lib\site-packages\theano\sandbox\cuda\__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


<module 'meta' from 'C:\\Users\\yolo\\Desktop\\fish\\meta.py'>

In [2]:
path_model_first_part = "models/ResNet_cut.h5"
folder_mid_vectors = "temp_training/resnet/"
path_model_heatmaps = "models/fish_detection_trained_1.1.h5"
folder_heatmaps = "heatmaps/tmp/"
final_folder = "./rectangles_scores/"

In [25]:
def store_scores2(heatmaps, folder, model, params_to_test, filename):
    fold = folder + filename
    utils.mk(fold)
    if len(os.listdir(fold)) != len(params_to_test):
        scores = model.predict(np.expand_dims(heatmaps,0))[0]
        for i, params in enumerate(params_to_test):
            score_filename = str(params[1]) + "_" + str(params[2])
            utils.save_array(scores[i], fold + "/" + score_filename)

In [26]:
def store_scores(metadata, path_model_first_part, folder_mid_vectors, path_model_heatmaps, 
                 folder_heatmaps,final_folder,  p_sizes = [(768, 1216),(1344,2240)], 
                 threshold=300, ranges=(11,71), border_conf=[(9,3),(71,10)]):
    
    p_meta = [(v["path"], v["filename"], v["height"], v["width"]) for key,v in metadata.items()][:200]
    
    # We make the intermidiary vectors
    pf.generate_intermediary_vectors(p_meta,  p_sizes, folder_mid_vectors, path_model_first_part)
        
    heatmaps = pf.create_heatmaps(p_meta, folder_mid_vectors, p_sizes, path_model_heatmaps, folder_heatmaps)
    
              
    conv_model , params_to_test= htr.create_model_rect(threshold, ranges, border_conf)
    # Finally we compute the scores:
    for i, (path, f_name, height, width) in enumerate(tq(p_meta)):
        store_scores2(heatmaps[i], final_folder, conv_model, params_to_test, f_name)
    
    utils.save(params_to_test, final_folder + "params_list.p")
    
    
    

In [27]:
store_scores(metadata, path_model_first_part, folder_mid_vectors, path_model_heatmaps, 
                 folder_heatmaps,final_folder,  p_sizes = [(768, 1216),(1344,2240)], 
                 threshold=300, ranges=(11,71), border_conf=[(9,3),(71,10)])

Exception in thread Thread-69:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 102, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Program Files\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

